## 配置文件

In [12]:
import sys
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
from risk_models.config.read_config.read_func import Read_Oracle
from risk_models.config.write_config.write_func import Write_Oracle
import datetime
import pandas as pd

## 库存库位盈亏模型

In [20]:
class StockModelSt1:
    def __init__(self, org_code, open_time, close_time):
        self.org_code = org_code
        self.open_time = open_time
        self.close_time = close_time
    
    def cleanData(self):
        # 分别读取期初库存head表/detail表（iscurrent = 1）
        STOCK_OPENING_HEAD = Read_Oracle().read_oracle(sql= """ select * from OPENING_INVENTORY where iscurrent = 1 """, database = 'dbods')
        STOCK_OPENING_DETAIL = Read_Oracle().read_oracle(sql= """ select * from OPENING_INVENTORY_DETAIL where iscurrent = 1 """, database = 'dbods')
        # 根据输入企业信用代码与开始时间，筛选期初库存表
        STOCK_OPENING_DETAIL_PID = int(STOCK_OPENING_HEAD[(STOCK_OPENING_HEAD['CREDIT_CODE'] == self.org_code) & (STOCK_OPENING_HEAD['OPT_DATE'] == self.open_time)]['ID'])
        STOCK_OPENING_DETAIL = STOCK_OPENING_DETAIL[STOCK_OPENING_DETAIL['PID'] == STOCK_OPENING_DETAIL_PID]
        # 转换“数量”为数字
        STOCK_OPENING_DETAIL['DCL_QTY'] = STOCK_OPENING_DETAIL['DCL_QTY'].map(float)
        # 汇总期初库存表
        STOCK_OPENING_DETAIL = STOCK_OPENING_DETAIL.groupby(['WAREHOUSE_NO','STORAGE_NO','GDS_MTNO'])['DCL_QTY'].sum().reset_index()
        STOCK_OPENING_DETAIL = STOCK_OPENING_DETAIL.rename(columns={'WAREHOUSE_NO':'WH_NO','STORAGE_NO':'WH_LOC','GDS_MTNO':'COP_G_NO'})
        
        # 读取出入库表（限制出入库类型）
        # ---- 测试用 ----
        #STOCK_BILL = Read_Oracle().read_oracle(sql= """ select * from ods_zmxpq.ems_stock_bill where CAPXACTION != 'D' and business_type in ('2','3','4') """, database='dbods')
        # ---- 生产用 ----
        STOCK_BILL = Read_Oracle().read_oracle(sql= """ select * from ods_zmxpq.ems_stock_bill where CAPXACTION != 'D' and bill_type in ('1','2','3','4','5','6','A','B') """, database='dbods')
        # 根据输入企业信用代码与起始时间，筛选出入库表
        STOCK_BILL = STOCK_BILL[(STOCK_BILL['ORG_CODE']==self.org_code) & (STOCK_BILL['ACTRUAL_STOCK_DATE']>=self.open_time) & (STOCK_BILL['ACTRUAL_STOCK_DATE']<=self.close_time)]
        # 转换“数量”为数字
        STOCK_BILL['QTY_CO'] = STOCK_BILL['QTY_CO'].map(float)
        # 转换“出入库类型”字段数据类型
        STOCK_BILL['STOCK_BILL_TYPE'] = STOCK_BILL['STOCK_BILL_TYPE'].map(int)
        # 对"出入库类型"进行转化便于计算
        STOCK_BILL['STOCK_BILL_TYPE'] = STOCK_BILL['STOCK_BILL_TYPE'].map(lambda x: -1 if x ==2 else 1)
        # 将出库的数量变为负数
        STOCK_BILL['QTY'] = STOCK_BILL['QTY'] * STOCK_BILL['STOCK_BILL_TYPE']
        # 汇总出入库表
        STOCK_BILL = STOCK_BILL.groupby(['WH_NO','WH_LOC','COP_G_NO'])['QTY_CO'].sum().reset_index()
        
        # 汇总期初和出入库表，生成期末库存表
        STOCK_END_DETAIL = pd.merge(STOCK_BILL, STOCK_OPENING_DETAIL, how='left', on = ['WH_NO','WH_LOC','COP_G_NO'])
        STOCK_END_DETAIL = STOCK_END_DETAIL.rename(columns={'QTY_CO':'QTY_CHANGE','DCL_QTY':'QTY_BEFORE'})
        # 将期初库存为空的行值设为0
        STOCK_END_DETAIL = STOCK_END_DETAIL.fillna(0)
        # 计算期末库存
        STOCK_END_DETAIL['QTY_AFTER'] = STOCK_END_DETAIL['QTY_CHANGE'] + STOCK_END_DETAIL['QTY_BEFORE']
        
        # 加入企业信息; 期初期末时间；ID; 模型运行时间
        STOCK_END_DETAIL['ORG_CODE'] = self.org_code
        STOCK_END_DETAIL['STARTDT'] = self.open_time
        STOCK_END_DETAIL['ENDDT'] = self.close_time
        STOCK_END_DETAIL['ID'] = range(len(STOCK_END_DETAIL))
        detail_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        STOCK_END_DETAIL['CHECK_TIME'] = datetime.datetime.strptime(detail_now, "%Y-%m-%d %H:%M:%S")
        
        # 重新排序ORG
        STOCK_END_DETAIL = STOCK_END_DETAIL[['ID','ORG_CODE','STARTDT','ENDDT','WH_NO','WH_LOC','COP_G_NO','QTY_BEFORE','QTY_CHANGE','QTY_AFTER','CHECK_TIME']]
        
        # 判断结果是否为空：为空只返回table，不为空写入数据库
        if STOCK_END_DETAIL.empty :
            return STOCK_END_DETAIL
        else:
            Write_Oracle().write_oracle('BD_RISK_DETAIL_STOCK_ST1',STOCK_END_DETAIL,org_code=self.org_code)
            return STOCK_END_DETAIL
    
    def calData(self):
        # 读取明细表
        STOCK_END_RESULT = Read_Oracle().read_oracle(sql= """ select * from BD_RISK_DETAIL_STOCK_ST1 where iscurrent = 1 """, database = 'dbods')
        # 筛选出制定企业的数据
        STOCK_END_RESULT = STOCK_END_RESULT[STOCK_END_RESULT['ORG_CODE'] == self.org_code]
        # 删去不用的列
        STOCK_END_RESULT.drop(columns = ['ID','CHECK_TIME','ISCURRENT','LASTUPDATE'],inplace=True)
        
        # 读取参数表(待更新)
        # parm = Read_Oracle().read_oracle(sql= """ select * from config_table """, database='dbods')
        # 设置参数
        # if 参数为null:
            # cutoff = {'高阈值': STOCK_END_RESULT['QTY_AFTER'].quantile(q=0.8), '低阈值':float(0)}
            # rate = {'盈':float(-50 / len(STOCK_END_RESULT)), '亏':float(-100 / len(STOCK_END_RESULT))}
        # else:
            # 读取参数表参数 
        
        # 根据期末库存值设定阈值
        cutoff = {'高阈值': STOCK_END_RESULT['QTY_AFTER'].quantile(q=0.8), '低阈值':float(0)}
        # 根据数据量设定惩罚系数（库存过高暂不扣分）
        ratio = {'盈':float(0), '亏':float(-100 / len(STOCK_END_RESULT))}
        
        # 通过阈值计算标签
        STOCK_END_RESULT['RISK_LABEL'] = STOCK_END_RESULT['QTY_AFTER'].map(lambda x: '库存过高' if x >= cutoff['高阈值'] else ('负库存' if x <= cutoff['低阈值'] else '正常'))
        # 计算分数
        STOCK_END_RESULT['SCORE'] = STOCK_END_RESULT['RISK_LABEL'].map(lambda x: ratio['盈'] if x == '库存过高' else (ratio['亏'] if x == '负库存' else 0))
        STOCK_END_SCORE = round(100 + STOCK_END_RESULT['SCORE'].sum(), 2)
        
        # 更新ID和运行时间
        STOCK_END_RESULT['ID'] = range(len(STOCK_END_RESULT))
        result_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        STOCK_END_RESULT['CHECK_TIME'] = datetime.datetime.strptime(result_now, "%Y-%m-%d %H:%M:%S")
        
        # 删去不用的列
        STOCK_END_RESULT.drop(columns = ['QTY_BEFORE','QTY_CHANGE','QTY_AFTER'],inplace=True)
        
        # 重新排序
        STOCK_END_RESULT = STOCK_END_RESULT[['ID','ORG_CODE','STARTDT','ENDDT','WH_NO','WH_LOC','COP_G_NO','RISK_LABEL','SCORE','CHECK_TIME']]
        
        # 判断结果是否为空：为空只返回table，不为空写入数据库
        if STOCK_END_RESULT.empty :
            return STOCK_END_RESULT, STOCK_END_SCORE
        else:
            Write_Oracle().write_oracle('BD_RISK_RESULT_STOCK_ST1',STOCK_END_RESULT,org_code = self.org_code)
            return STOCK_END_RESULT, STOCK_END_SCORE
        

## 测试对象（参数：企业信用代码；期初期末时间）

In [21]:
STOCK_BIN_Model = StockModelSt1('91310000132612172J',datetime.datetime.strptime('2020-01-01', "%Y-%m-%d"),datetime.datetime.strptime('2021-12-31', "%Y-%m-%d"))

In [22]:
STOCK_END_DETAIL = STOCK_BIN_Model.cleanData()
STOCK_END_RESULT, STOCK_END_SCORE = STOCK_BIN_Model.calData()

2021-04-30 15:21:53.559 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.171s
2021-04-30 15:21:57.177 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 3.614s
2021-04-30 15:22:09.607 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 11.85s
2021-04-30 15:22:11.744 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.093s
2021-04-30 15:22:11.992 | INFO     | risk_models.config.write_config.write_func:write_oracle:128 - Processing... Writing 1635 rows into database
2021-04-30 15:22:15.057 | INFO     | risk_models.config.write_config.write_func:write_oracle:135 - Insert data into BD_RISK_DETAIL_STOCK_ST1 successfully! Total write time spent 3.411s
2021-04-30 15:22:15.196 | INFO     | risk_models.config.read_confi

## 明细表

In [23]:
STOCK_END_DETAIL

,ID,ORG_CODE,STARTDT,ENDDT,WH_NO,WH_LOC,COP_G_NO,QTY_BEFORE,QTY_CHANGE,QTY_AFTER,CHECK_TIME
0,0,91310000132612172J,2020-01-01,2021-12-31,1001,D001D0000000,000000310000003761,0.0,1.0,1.0,2021-04-30 15:22:11
1,1,91310000132612172J,2020-01-01,2021-12-31,1001,D001D0000001,000000310000005426,0.0,2.0,2.0,2021-04-30 15:22:11
2,2,91310000132612172J,2020-01-01,2021-12-31,1001,D001D0000003,000000310000003284,0.0,9.0,9.0,2021-04-30 15:22:11
3,3,91310000132612172J,2020-01-01,2021-12-31,1001,D001D0000003,000000310000003473,0.0,3.0,3.0,2021-04-30 15:22:11
4,4,91310000132612172J,2020-01-01,2021-12-31,1001,D001D0000003,000000310000005686,0.0,2.0,2.0,2021-04-30 15:22:11
...,...,...,...,...,...,...,...,...,...,...,...
1630,1630,91310000132612172J,2020-01-01,2021-12-31,1123,WPDFA0010101,000000310000005153,0.0,1.0,1.0,2021-04-30 15:22:11
1631,1631,91310000132612172J,2020-01-01,2021-12-31,1123,WPDFA0010101,000000310000005154,0.0,2.0,2.0,2021-04-30 15:22:11
1632,1632,91310000132612172J,2020-01-01,2021-12-31,1123,WPDFA0010101,000000310000005156,0.0,1.0,1.0,2021-04-30 15:22:11
1633,1633,91310000132612172J,2020-01-01,2021-12-31,1138,P203N0010101,000000110000005114,0.0,10.0,10.0,2021-04-30 15:22:11


## 结果表

In [24]:
STOCK_END_RESULT

,ID,ORG_CODE,STARTDT,ENDDT,WH_NO,WH_LOC,COP_G_NO,RISK_LABEL,SCORE,CHECK_TIME
0,0,91310000132612172J,2020-01-01,2021-12-31,1002,D001Q0130402,000000210000013573,正常,0.0,2021-04-30 15:22:15
1,1,91310000132612172J,2020-01-01,2021-12-31,1002,D001Q0130408,000000210000009025,正常,0.0,2021-04-30 15:22:15
2,2,91310000132612172J,2020-01-01,2021-12-31,1002,D001Q0130602,000000210000017500,正常,0.0,2021-04-30 15:22:15
3,3,91310000132612172J,2020-01-01,2021-12-31,1002,D001Q0140113,000000210000001862,正常,0.0,2021-04-30 15:22:15
4,4,91310000132612172J,2020-01-01,2021-12-31,1002,D001Q0140222,000000210000001265,库存过高,0.0,2021-04-30 15:22:15
...,...,...,...,...,...,...,...,...,...,...
1630,1630,91310000132612172J,2020-01-01,2021-12-31,1123,WPDFA0010101,000000310000005153,正常,0.0,2021-04-30 15:22:15
1631,1631,91310000132612172J,2020-01-01,2021-12-31,1123,WPDFA0010101,000000310000005154,正常,0.0,2021-04-30 15:22:15
1632,1632,91310000132612172J,2020-01-01,2021-12-31,1123,WPDFA0010101,000000310000005156,正常,0.0,2021-04-30 15:22:15
1633,1633,91310000132612172J,2020-01-01,2021-12-31,1138,P203N0010101,000000110000005114,正常,0.0,2021-04-30 15:22:15


## 分数

In [25]:
STOCK_END_SCORE

100.0